In [1]:
import csv
import h5py
import random
import requests
import numpy as np
import scipy.integrate
from hoki.constants import *
import illustris_python as il
import matplotlib.pyplot as plt

In [2]:
def get(path, params=None):
    r = requests.get(path, params=params, headers={"api-key":"ac0fbfe1251d6f93c582c5490527df5d"})
    r.raise_for_status()  # raise exception if the request does not succeed (does not get HTTP code 200)
    if r.headers['content-type'] == 'application/json':
        return r.json() # parse json responses automatically
    if 'content-disposition' in r.headers:
        filename = r.headers['content-disposition'].split("filename=")[1]
        with open(filename, 'wb') as f:
            f.write(r.content)
        return filename # return the filename string
    return r

In [ ]:
r = get('http://www.tng-project.org/api/')
sim = get(r['simulations'][6]['url'])
snaps = get(sim['snapshots'])  # retrieves snapshot information (snapshot number, redshift, number of subhalos, and url)
snap = get(snaps[99]['url'])  # retrieves global information for snapshot 99 in TNG100-1
subhalos = get(snap['subhalos'], {'limit':894940, 'order_by':'-mass'})  # retrieves subhalo information (list of [limit:num] subhalos)

In [ ]:
print(get(subhalos['results'][290668]['url']))  # finds exact info of subhalo

In [4]:
Z_lower = [0.00000, 0.00005, 0.0005, 0.0015, 0.0025, 0.0035, 0.005, 0.007, 0.009, 0.012, 0.017, 0.025, 0.035]
Z_upper = [0.00005, 0.00050, 0.0015, 0.0025, 0.0035, 0.0050, 0.007, 0.009, 0.012, 0.017, 0.025, 0.035, 10.00]

In [5]:
mass4_Z040 = []
mass4_Z030 = []
mass4_Z020 = []
mass4_Z014 = []
mass4_Z010 = []
mass4_Z008 = []
mass4_Z006 = []
mass4_Z004 = []
mass4_Z003 = []
mass4_Z002 = []
mass4_Z001 = []
mass4_Zem4 = []
mass4_Zem5 = []

In [ ]:
for subhaloID in np.linspace(13797, 99678, 85881, endpoint=False):
    subhalo = get(subhalos['results'][int(subhaloID)]['url'])
    if 1 <= subhalo['mass'] < 10:
        if Z_lower[0] <= subhalo['gasmetallicitysfrweighted'] < Z_upper[0]:
            mass4_Zem5.append(subhalo['id'])
        if Z_lower[1] <= subhalo['gasmetallicitysfrweighted'] < Z_upper[1]:
            mass4_Zem4.append(subhalo['id'])
        if Z_lower[2] <= subhalo['gasmetallicitysfrweighted'] < Z_upper[2]:
            mass4_Z001.append(subhalo['id'])
        if Z_lower[3] <= subhalo['gasmetallicitysfrweighted'] < Z_upper[3]:
            mass4_Z002.append(subhalo['id'])
        if Z_lower[4] <= subhalo['gasmetallicitysfrweighted'] < Z_upper[4]:
            mass4_Z003.append(subhalo['id'])
        if Z_lower[5] <= subhalo['gasmetallicitysfrweighted'] < Z_upper[5]:
            mass4_Z004.append(subhalo['id'])            
        if Z_lower[6] <= subhalo['gasmetallicitysfrweighted'] < Z_upper[6]:
            mass4_Z006.append(subhalo['id'])            
        if Z_lower[7] <= subhalo['gasmetallicitysfrweighted'] < Z_upper[7]:
            mass4_Z008.append(subhalo['id'])
        if Z_lower[8] <= subhalo['gasmetallicitysfrweighted'] < Z_upper[8]:
            mass4_Z010.append(subhalo['id'])            
        if Z_lower[9] <= subhalo['gasmetallicitysfrweighted'] < Z_upper[9]:
            mass4_Z014.append(subhalo['id'])            
        if Z_lower[10] <= subhalo['gasmetallicitysfrweighted'] < Z_upper[10]:
            mass4_Z020.append(subhalo['id'])            
        if Z_lower[11] <= subhalo['gasmetallicitysfrweighted'] < Z_upper[11]:
            mass4_Z030.append(subhalo['id'])            
        if Z_lower[12] <= subhalo['gasmetallicitysfrweighted'] < Z_upper[12]:
            mass4_Z040.append(subhalo['id'])            

In [ ]:
print(len(mass4_Z040), len(mass4_Z030), len(mass4_Z020), len(mass4_Z014), len(mass4_Z010), len(mass4_Z008), len(mass4_Z006), len(mass4_Z004), 
      len(mass4_Z003), len(mass4_Z002), len(mass4_Z001), len(mass4_Zem4), len(mass4_Zem5))

In [ ]:
basePath = './sims.TNG/TNG100-1/postprocessing'
fields = ['SubfindID', 'SnapNum']
tree = il.sublink.loadTree(basePath, 99, 809864, fields=fields)
subfindIDs = tree['SubfindID']
snapnums = tree['SnapNum']

subhalo_sfr = np.empty(len(snapnums))
for snapshot in range(0, 100):
    path = './sims.TNG/TNG100-1/output/'
    fields = ['SubhaloSFR', 'SubhaloGasMetallicitySfrWeighted', 'SubhaloMass']
    subhalo = il.groupcat.loadSubhalos(path, snapshot, fields=fields)
    for index, snapnum in enumerate(snapnums):
        if snapnum == snapshot:
            subhalo_sfr[index] = subhalo['SubhaloMass'][subfindIDs[index]]
    print(snapshot)
    
sfr = []
for snapshot in range(0, 100):
    sum_sfr = []
    for index, snapnum in enumerate(snapnums):
        if snapnum == snapshot:
            sum_sfr.append(subhalo_sfr[index])
    sfr.append(np.sum(sum_sfr))
    
file = open(f'./mass7_Z030', 'w')
writer = csv.writer(file)
writer.writerow(sfr)
file.close()